## Do not change this notebooks 
We use it for preprocessing

In [ ]:
import sys
import os 
sys.path.insert(1, os.path.realpath(os.path.pardir))

import numpy as np
from pathlib import Path
from natsort import natsorted

#import wandb
import torch
# import torch.nn as nn
# import torch.nn.functional as F

# import pytorch_lightning as pl
# from pytorch_lightning.loggers import WandbLogger
# from pytorch_lightning.callbacks import Callback, ModelCheckpoint
# from pytorch_model_summary import summary

from utils import data_utils
from utils import losses, hand_visualize

from utils.quats_and_angles import get_angles, get_quats
from einops import rearrange
from tqdm import tqdm
from scipy import signal
import matplotlib.pyplot as plt

In [2]:
%matplotlib qt

In [3]:
def butter_highpass(cutoff, nyq_freq, order=4):
    normal_cutoff = float(cutoff) / nyq_freq
    b, a = signal.butter(order, normal_cutoff, btype='highpass')
    return b, a

def butter_highpass_filter(data, cutoff_freq, nyq_freq, order=4):
    b, a = butter_highpass(cutoff_freq, nyq_freq, order=order)
    y = signal.filtfilt(b, a, data)
    return y

In [2]:
config = dict(  
    original_fps = 200,  # TODO describtion
    delay_ms = 0,  # Shift vr vs EMG parameter. ms dealy between emg and VR.
    start_crop_ms = 0,  # bad values in the beginning of recordign in ms to delete.
    window_size = 256,
)


class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
config = AttrDict(config)

    
def get_subdirs(rootdir):
    paths = []
    for path in Path(rootdir).iterdir():
        if path.is_dir():
            paths.append(path)
    return paths

# Get paths and preprocess raw datasets

In [3]:
ROOT_DIR = Path("D:\\Myo_Project\\MIO_trindets\\Subj17\\")

DATASET_NAMES = ['Raw']

In [4]:
'''aaa= [1,2,3,4]
print(aaa[:3])

print(ALL_PATHS)

print(path)'''

'aaa= [1,2,3,4]\nprint(aaa[:3])\n\nprint(ALL_PATHS)\n\nprint(path)'

In [ ]:
# get all paths to all folders inside above datasets 
Nch_target = 6

dataset_paths = [ROOT_DIR / Path(n) for n in DATASET_NAMES]
print(dataset_paths)
ALL_PATHS = []
for dp in dataset_paths:
    ALL_PATHS.extend(get_subdirs(dp))

print('ALL_PATHS: ', ALL_PATHS)
print('Number of paths: ', len(ALL_PATHS))


# Preprocess all datasets for angles extraction.

for path in ALL_PATHS:
    sorted_pth = sorted(path.glob('*.npz'))   
    sorted_pth= natsorted(sorted_pth)
    
    
    #if use_preproc_data == True:
    #    exps_data = [dict(np.load(d)) for d in all_paths]
        

    dataset = data_utils.create_dataset(data_folder=path,
                                              original_fps=config.original_fps,
                                              delay_ms=config.delay_ms,
                                              start_crop_ms=config.start_crop_ms,
                                              window_size=config.window_size,
                                              random_sampling=False,
                                              transform=None)
    
    
    if len(dataset)==0: 
        print('WWWWW: Problem with dataset')
        break\


    my_myo_list = list()
    

    # go through each move and get angles and save.
    for idx, move in tqdm(enumerate(dataset.exps_data)):

        my_myo =  np.load(sorted_pth[idx])['data_myo']
        my_myo = my_myo[: len(move['data_myo'])]

        my_myo_list.append(my_myo)

        std_coef = np.std(my_myo,axis = 0,keepdims = True)
        print(std_coef)


    my_myo_list = np.concatenate(my_myo_list) 
    std_coef = np.std(my_myo_list,axis = 0,keepdims = True)    


    #std_coef_1 = my_myo_list[:len(my_myo_list)//2]
    #std_coef_2 = my_myo_list[len(my_myo_list)//2:]

    #print(std_coef_1,std_coef_2)
    
    
    
    # go through each move and get angles and save.
    for idx, move in tqdm(enumerate(dataset.exps_data)):

        my_myo =  np.load(sorted_pth[idx])['data_myo']

        
        my_myo_norm = my_myo[:,:Nch_target]/std_coef[:,:Nch_target]

        my_myo_norm = my_myo_norm[: len(move['data_myo'])]
        
        
        ts, myo, vr = move['myo_ts'], my_myo_norm, move['data_vr']
        angles = get_angles(vr)
        
        new_path = path.parents[0] / Path('preproc_angles') / Path(path.name)
        new_path.mkdir(parents=True, exist_ok=True)
        
        filename = f"{idx:04d}.npz"
        filepath = new_path / filename
        
        np.savez(filepath, data_myo=myo,
                 data_vr=vr, data_angles=angles, myo_ts=ts, std_coef = std_coef)


    

[WindowsPath('D:/Myo_Project/MIO_trindets/Subj17/Raw')]
ALL_PATHS:  [WindowsPath('D:/Myo_Project/MIO_trindets/Subj17/Raw/1')]
Number of paths:  1
Number of moves: 8 | Dataset: Subj17


  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

Slice myo_timestamps and all data from 0 to 14454


 12%|██████████▌                                                                         | 1/8 [00:03<00:23,  3.42s/it]

Slice myo_timestamps and all data from 0 to 14455


...

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:25<00:00,  3.15s/it]


Total len: 448


8it [00:00, 402.35it/s]


[[1.62301220e-04 2.20848369e-04 5.83913470e-05 1.92107188e-04
  9.47738925e-05 6.62664909e-05 0.00000000e+00 0.00000000e+00]]
[[1.73728193e-04 2.29962913e-04 4.79786292e-05 1.75272082e-04
  1.09155634e-04 4.82419294e-05 0.00000000e+00 0.00000000e+00]]
[[1.06989355e-04 1.42752047e-04 6.15879231e-05 8.33903478e-05
  1.22122290e-04 3.28982901e-05 0.00000000e+00 0.00000000e+00]]
[[3.13996337e-04 2.41188662e-04 1.18007489e-04 2.39755317e-04
  3.41600863e-04 8.91925402e-05 0.00000000e+00 0.00000000e+00]]
[[2.23319099e-05 7.65822405e-05 2.21113007e-05 5.47354383e-05
  2.18146146e-05 2.79627457e-05 0.00000000e+00 0.00000000e+00]]
[[1.63967373e-04 1.99753112e-04 6.28982404e-05 1.64563452e-04
  1.04386716e-04 6.00055142e-05 0.00000000e+00 0.00000000e+00]]
[[6.90823197e-05 1.19120184e-04 5.56012699e-05 6.26676582e-05
  7.42049511e-05 3.83179619e-05 0.00000000e+00 0.00000000e+00]]
[[1.54631170e-05 3.68411671e-05 1.68666321e-05 5.46903241e-05
  1.71748899e-05 1.05991504e-05 0.00000000e+00 0.0000000

8it [01:56, 14.54s/it]


# Check the quality. How to use new data.

Here I visualize and check correctness of angle prediction. 

In [13]:
name = 'test'
subj = 'Vlad'

In [42]:
rootdir = Path(f'C:/Users/vlvdi/Desktop/EMG/MYO_DATA/{subj}/GeneralTraining/{name}/preproc_angles')
#rootdir = Path(f'C:/Users/vlvdi/Desktop/EMG/Nastya/10_electrodes_28022024/preproc_angles')
files = list(rootdir.glob('*/*'))
data = np.load(files[5])

emg, quats, angles = data['data_myo'], data['data_vr'], data['data_angles']

In [43]:
plt.plot((np.sum(angles, axis=1) - 1) * 0.2)
plt.plot(emg[:, 2], alpha=0.6)

In [ ]:
signal = emg[:, 4]

In [ ]:
files[3]

In [ ]:
rootd = Path(f'C:/Users/vlvdi/Desktop/EMG/Nastya/10_electrodes_28022024/preproc_angles/1/0001.npz')

In [ ]:
dd = np.load(rootd)
emg, ang = dd['data_myo'], dd['data_vr']

In [ ]:
plt.plot(emg[:, 7], alpha=0.8)

In [ ]:
sig = emg[:, 7]

In [ ]:
sig = sig[~np.isnan(sig)]

In [ ]:
samp_freq = 500  # Sample frequency (Hz)
notch_freq = 50.0  # Frequency to be removed from signal (Hz)
quality_factor = 30.0  # Quality factor

In [ ]:
b_notch, a_notch = signal.iirnotch(notch_freq, quality_factor, samp_freq)
freq, h = signal.freqz(b_notch, a_notch, fs = samp_freq)

In [ ]:
y_notched = signal.filtfilt(b_notch, a_notch, sig)

In [ ]:
plt.plot(y_notched)

In [ ]:
from utils.hand_visualize import Hand, save_animation_mp4, visualize_and_save_anim, merge_two_videos, visualize_and_save_anim_gifs

def get_angle_degree(y_hat, y_batch):
    """
    [batch, n_bones, 4, time]
    """
    time, n_bones, n_quat,  = y_hat.shape
    y_hat, y_batch = y_hat.reshape(-1, 4), y_batch.reshape(-1, 4)

    mult = torch.sum(y_hat*y_batch, dim=-1)**2
    angle_degree = torch.mean(torch.arccos(torch.clip((2*mult -1), -1, 1))/torch.pi*180)
    return angle_degree.item()

In [ ]:
rootdir = Path('C:/Users/vlvdi/Desktop/EMG/oleg500/preproc_angles')
files = list(rootdir.glob('*/*'))
data = np.load(files[0])

emg, quats, angles = data['data_myo'], data['data_vr'], data['data_angles']
quats_hat = get_quats(angles)
angle_diff = get_angle_degree(torch.from_numpy(quats_hat), torch.from_numpy(quats))

print('Shapes', emg.shape, quats.shape, angles.shape, quats_hat.shape)
print('Angle degree', angle_diff)

In [ ]:
(0.0002 + 128) / 255

In [ ]:
0.501961568627451 * 255 - 128

In [ ]:
import scipy.signal as signal

In [ ]:
emg_filt = butter_highpass_filter(emg[:, 2], 5, 125, order=5)

In [ ]:
plt.plot(emg_filt[:])

In [ ]:
plt.plot(angles[:10_000, :])
plt.plot((emg_filt[:10_000]) * 200000, alpha=0.8)

In [ ]:
quats.shape

In [ ]:
NEW_FPS = 25
DRAW_EVERY = 250//25

visualize_and_save_anim_gifs(data=quats[:2000:DRAW_EVERY],
                        path=Path('C:/Users/vlvdi/Desktop/EMG/11_15_2023/original_quats.gif'), 
                        fps=NEW_FPS)

In [ ]:
visualize_and_save_anim_gifs(data=quats_hat[:2000:DRAW_EVERY],
                        path=Path('C:/Users/vlvdi/Desktop/EMG/11_15_2023/from_angles_quats.gif'), 
                        fps=NEW_FPS)